In [3]:
# Data file at https://www.cse.ust.hk/msbd5003/data

lines = sc.textFile('/Users/zhaocai/adj_noun_pairs.txt', 8)

In [4]:
lines.count()

3162692

In [5]:
lines.getNumPartitions()

8

In [6]:
lines.take(5)

['early radical',
 'french revolution',
 'pejorative way',
 'violent means',
 'positive label']

In [7]:
# Converting lines into word pairs. 
# Data is dirty: some lines have more than 2 words, so filter them out.
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

PythonRDD[6] at RDD at PythonRDD.scala:49

In [8]:
pairs.take(5)

[('early', 'radical'),
 ('french', 'revolution'),
 ('pejorative', 'way'),
 ('violent', 'means'),
 ('positive', 'label')]

In [9]:
N = pairs.count()

In [10]:
N

3162674

In [11]:
# Compute the frequency of each pair.
# Ignore pairs that not frequent enough
pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2) \
                  .filter(lambda pf: pf[1] >= 100)

In [12]:
pair_freqs.take(5)

[(('16th', 'century'), 950),
 (('civil', 'war'), 2236),
 (('social', 'class'), 155),
 (('sixteenth', 'century'), 137),
 (('late', '1970'), 444)]

In [13]:
# Computing the frequencies of the adjectives and the nouns
a_freqs = pairs.map(lambda p: (p[0],1)).reduceByKey(lambda x,y: x+y)
n_freqs = pairs.map(lambda p: (p[1],1)).reduceByKey(lambda x,y: x+y)

In [14]:
a_freqs.take(5)

[('social', 7413),
 ('free', 6635),
 ('stoic', 72),
 ('good', 7499),
 ('other', 75260)]

In [26]:
n_freqs.count()

106333

In [30]:
# Broadcasting the adjective and noun frequencies. 
#a_dict = a_freqs.collectAsMap()
#a_dict = sc.parallelize(a_dict).map(lambda x: x)
#n_dict = sc.broadcast(n_freqs.collectAsMap())
#a_dict = sc.broadcast(a_freqs.collectAsMap())
#a_dict.value['violent']
#scored_pairs = pair_freqs.map(lambda x:(log(float(x[1])*N/(a_freqs[x[0][0]]*n_freqs[x[0][1]]), 2), x[0][0],x[0][1]))
#n_dict = n_freqs.collectAsMap()
#a_dict = a_freqs.collectAsMap()
#scored_pairs = pair_freqs.map(lambda x: (x[1]*N, x[0][0],x[0][1]))
#scored_pairs = pair_freqs.map(lambda x: (float(x[1])*N/(a_freqs[x[0][0]]*n_freqs[x[0][1]]), x[0][0],x[0][1]))
#scored_pairs = pair_freqs.join(n_freqs,)
#scored_pairs.collect()
a_dict = a_freqs.collectAsMap()
a_dict = sc.parallelize(a_dict).map(lambda x: x)
n_dict = n_freqs.collectAsMap()
n_dict = sc.parallelize(n_dict).map(lambda x: x)

In [31]:
from math import *

# Computing the PMI for a pair.
def pmi_score(pair_freq):
    w1, w2 = pair_freq[0]
    f = pair_freq[1]
    pmi = log(float(f)*N/(a_dict.value[w1]*n_dict.value[w2]), 2)
    return pmi, (w1, w2)

In [32]:
# Computing the PMI for all pairs.
scored_pairs = pair_freqs.map(pmi_score)

In [33]:
# Printing the most strongly associated pairs. 
scored_pairs.top(10)

Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.1/libexec/python/pyspark/cloudpickle.py", line 235, in dump
    return Pickler.dump(self, obj)
  File "/anaconda3/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/anaconda3/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/local/Cellar/apache-spark/2.3.1/libexec/python/pyspark/cloudpickle.py", line 378, in save_function
    self.save_function_tuple(obj)
  File "/usr/local/Cellar/apache-spark/2.3.1/libexec/python/pyspark/cloudpickle.py", line 529, in save_function_tuple
    save(closure_values)
  File "/anaconda3/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File

PicklingError: Could not serialize object: Exception: It appears that you are attempting to broadcast an RDD or reference an RDD from an action or transformation. RDD transformations and actions can only be invoked by the driver, not inside of other transformations; for example, rdd1.map(lambda x: rdd2.values.count() * x) is invalid because the values transformation and count action cannot be performed inside of the rdd1.map transformation. For more information, see SPARK-5063.

In [1]:
products = {1: "Apple", 2: "Orange", 3: "TV", 5: "Computer"}
trans = sc.parallelize([(1, (134, "OK")), (3, (34, "OK")), (5, (162, "Error")), (1, (135, "OK")), (2, (53, "OK")), (1, (45, "OK"))])

# broadcasted_products = sc.broadcast(products)

def f(x):
    return (x[0], broadcasted_products.value[x[0]], x[1])

# results = trans.map(lambda x: (x[0], broadcasted_products.value[x[0]], x[1]))
results = trans.map(lambda x: (x[0], products[x[0]], x[1]))
print (results.collect())

[(1, 'Apple', (134, 'OK')), (3, 'TV', (34, 'OK')), (5, 'Computer', (162, 'Error')), (1, 'Apple', (135, 'OK')), (2, 'Orange', (53, 'OK')), (1, 'Apple', (45, 'OK'))]
